In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, date

from sklearn.model_selection import train_test_split

from pathlib import Path

import holidays

from mlforecast import MLForecast
from mlforecast.lag_transforms import RollingMean, RollingStd
from lightgbm import LGBMRegressor

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

In [2]:
data_dir = Path("../Test-Task-for-DS-time-series-forecasting-2026-01/data")
prepared = data_dir / "data_prepared.csv"

print("prepared:", prepared.exists())

prepared: True


### Some prepare

In [3]:
df = pd.read_csv(prepared)
print(df.shape, "\n")
df.head()

(350730, 12) 



,unique_id,ds,y,is_observed,day_of_week,month,day_of_month,week_of_year,is_weekend,holiday_type,is_any_holiday,all_holiday_name
0,0_FOODS_1_0,2014-01-01,23,1,2,1,1,1,0,National,1,NewYear | New Year's Day
1,0_FOODS_1_0,2014-01-02,28,1,3,1,2,1,0,NaN,0,NaN
2,0_FOODS_1_0,2014-01-03,43,1,4,1,3,1,0,NaN,0,NaN
3,0_FOODS_1_0,2014-01-04,33,1,5,1,4,1,1,NaN,0,NaN
4,0_FOODS_1_0,2014-01-05,32,1,6,1,5,1,1,NaN,0,NaN


In [4]:
df['ds'] = pd.to_datetime(df['ds'])
df["all_holiday_name"] = df["all_holiday_name"].fillna("None")
df["holiday_type"] = df["holiday_type"].fillna("None")
df['unique_id'] = df['unique_id'].astype('category')
df['holiday_type'] = df['holiday_type'].astype('category')
df['all_holiday_name'] = df['all_holiday_name'].astype('category')
df[["holiday_type", "all_holiday_name"]].isna().sum()
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 350730 entries, 0 to 350729
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   unique_id         350730 non-null  category      
 1   ds                350730 non-null  datetime64[us]
 2   y                 350730 non-null  int64         
 3   is_observed       350730 non-null  int64         
 4   day_of_week       350730 non-null  int64         
 5   month             350730 non-null  int64         
 6   day_of_month      350730 non-null  int64         
 7   week_of_year      350730 non-null  int64         
 8   is_weekend        350730 non-null  int64         
 9   holiday_type      350730 non-null  category      
 10  is_any_holiday    350730 non-null  int64         
 11  all_holiday_name  350730 non-null  category      
dtypes: category(3), datetime64[us](1), int64(8)
memory usage: 25.4 MB


## MLForecast + LightGBM

### Validation

In [ ]:
cut_off_date = pd.Timestamp('2016-05-08') 
train_df = df[df['ds'] <= cut_off_date].copy()
valid_df = df[(df['ds'] > cut_off_date) & (df['ds'] <= pd.Timestamp('2016-05-15'))].copy()

In [30]:
lgbm = LGBMRegressor(n_estimators=1000, 
                     learning_rate=0.05,
                     num_leaves=64,
                     colsample_bytree=0.8,
                     subsample=0.9,
                     random_state=42,
                     objective='poisson'
                    )

lags = [1, 2, 3, 7, 14, 21, 28, 35, 56]

lag_transforms = {1: 
                  [
                      RollingMean(3),
                      RollingMean(7),
                      RollingMean(28),
                      RollingMean(56),
                      RollingStd(7),

                  ]
            }


In [31]:
X_valid = valid_df.drop(columns=['y'])

frct = MLForecast(
    models=[lgbm],
    lags=lags,
    lag_transforms=lag_transforms,
    freq='D'
)
frct.fit(train_df, 
         id_col='unique_id',
         time_col='ds', 
         target_col='y', 
         static_features=[]
        )

pred_valid = frct.predict(h=7, X_df=X_valid)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006403 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3052
[LightGBM] [Info] Number of data points in the train set: 325215, number of used features: 23
[LightGBM] [Info] Start training from score 1.796603


In [32]:
pred_col = [c for c in pred_valid.columns if c not in ['unique_id', 'ds']][0]
check = valid_df.merge(pred_valid, on=['unique_id', 'ds'], how='inner').copy()

check['y_pred'] = check[pred_col].clip(lower=0).round().astype('int64')

weekly = (check
          .groupby('unique_id', as_index=False)
          .agg(y_true=('y', 'sum'),
               y_pred=('y_pred', 'sum')))

eps = 1e-8
weekly['ape'] = (weekly['y_true'] - weekly['y_pred']).abs() / np.maximum(weekly['y_true'].abs(), eps)

mape_weekly = weekly['ape'].mean() * 100
print("Weekly MAPE (%):", mape_weekly)
print("How many series:", weekly['unique_id'].nunique(), "rows:", len(weekly))


Weekly MAPE (%): 22.502683188156194
How many series: 405 rows: 405


### OPTUNA

In [41]:
cut_off_date = pd.Timestamp('2016-05-08') 
train_df = df[df['ds'] <= cut_off_date].copy()
valid_df = df[(df['ds'] > cut_off_date) & (df['ds'] <= pd.Timestamp('2016-05-15'))].copy()

In [42]:
# optuna
def weekly_mape(y_true, y_pred, eps=1e-8):
    mask = y_true > 0
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    # MAPE по 405 рядам (тижнева сума). Якщо є нулі — захистимося eps.
    return np.mean(np.abs(y_true - y_pred) / np.maximum(np.abs(y_true), eps)) * 100

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 2000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 10.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 10.0),
        "random_state": 42,
        "objective": "poisson",
        "n_jobs": -1,
        "verbosity": -1,
    }

    lgbm_opt = LGBMRegressor(**params)
    frct_opt = MLForecast(
        models=[lgbm_opt],
        lags=lags,
        lag_transforms=lag_transforms,
        freq='D',
        )
    
    frct_opt.fit(train_df, id_col='unique_id', time_col='ds', target_col='y', static_features=[])

    X_valid = valid_df.drop(columns=['y'])
    pred = frct_opt.predict(h=7, X_df=X_valid)

    pred_col = [c for c in pred.columns if c not in ['unique_id', 'ds']][0]

    y_pred_week = pred.groupby('unique_id')[pred_col].sum()
    y_true_week = valid_df.groupby('unique_id')['y'].sum()

    y_true_week = y_true_week.reindex(y_pred_week.index)

    return weekly_mape(y_true_week.values, y_pred_week.values)

In [43]:
study = optuna.create_study(
    direction="minimize",  # MAPE: менше = краще
    sampler=TPESampler(seed=42),
    pruner=MedianPruner(n_startup_trials=10, n_warmup_steps=0),
)

study.optimize(objective, n_trials=20, show_progress_bar=True)

print("Best weekly MAPE (%):", study.best_value)
print("Best params:", study.best_params)


[I 2026-01-29 15:51:28,975] A new study created in memory with name: no-name-f68d8757-442e-43e2-af2f-c531b9e2c475
Best trial: 0. Best value: 24.5573:   5%|▌         | 1/20 [00:20<06:22, 20.13s/it]

[I 2026-01-29 15:51:49,107] Trial 0 finished with value: 24.557307597158403 and parameters: {'n_estimators': 937, 'learning_rate': 0.09556428757689246, 'num_leaves': 195, 'min_data_in_leaf': 307, 'feature_fraction': 0.6624074561769746, 'bagging_fraction': 0.662397808134481, 'bagging_freq': 1, 'lambda_l1': 8.661761457749352, 'lambda_l2': 6.011150117432088}. Best is trial 0 with value: 24.557307597158403.


Best trial: 1. Best value: 24.5097:  10%|█         | 2/20 [00:56<08:52, 29.58s/it]

[I 2026-01-29 15:52:25,308] Trial 1 finished with value: 24.509719640198384 and parameters: {'n_estimators': 1504, 'learning_rate': 0.011852604486622221, 'num_leaves': 249, 'min_data_in_leaf': 420, 'feature_fraction': 0.6849356442713105, 'bagging_fraction': 0.6727299868828402, 'bagging_freq': 2, 'lambda_l1': 3.0424224295953772, 'lambda_l2': 5.247564316322379}. Best is trial 1 with value: 24.509719640198384.


Best trial: 2. Best value: 24.1377:  15%|█▌        | 3/20 [01:16<07:12, 25.46s/it]

...

[I 2026-01-29 15:57:36,283] Trial 17 finished with value: 24.326599489091503 and parameters: {'n_estimators': 1681, 'learning_rate': 0.05628341461329368, 'num_leaves': 122, 'min_data_in_leaf': 239, 'feature_fraction': 0.8505915654206465, 'bagging_fraction': 0.7027131155204999, 'bagging_freq': 9, 'lambda_l1': 6.3618570164385915, 'lambda_l2': 9.971564417052052}. Best is trial 12 with value: 24.06997758286577.


Best trial: 12. Best value: 24.07:  95%|█████████▌| 19/20 [06:20<00:17, 17.71s/it]

[I 2026-01-29 15:57:49,049] Trial 18 finished with value: 24.16890886894828 and parameters: {'n_estimators': 648, 'learning_rate': 0.08162809857042158, 'num_leaves': 121, 'min_data_in_leaf': 187, 'feature_fraction': 0.7290942141364104, 'bagging_fraction': 0.6631805200240225, 'bagging_freq': 6, 'lambda_l1': 4.6012610210892895, 'lambda_l2': 7.9335280400443295}. Best is trial 12 with value: 24.06997758286577.


Best trial: 12. Best value: 24.07: 100%|██████████| 20/20 [06:37<00:00, 19.89s/it]

[I 2026-01-29 15:58:06,836] Trial 19 finished with value: 24.289924939593238 and parameters: {'n_estimators': 1178, 'learning_rate': 0.09882474708640576, 'num_leaves': 60, 'min_data_in_leaf': 496, 'feature_fraction': 0.7640881690357239, 'bagging_fraction': 0.6020099588015292, 'bagging_freq': 9, 'lambda_l1': 2.0601962491467956, 'lambda_l2': 6.480206973232005}. Best is trial 12 with value: 24.06997758286577.
Best weekly MAPE (%): 24.06997758286577
Best params: {'n_estimators': 537, 'learning_rate': 0.09617540281067662, 'num_leaves': 90, 'min_data_in_leaf': 26, 'feature_fraction': 0.7874820875551369, 'bagging_fraction': 0.7163212943395332, 'bagging_freq': 7, 'lambda_l1': 0.33585884934472254, 'lambda_l2': 7.49778813154158}


### Prediction

In [19]:
exog_cols = [
    # 'is_observed',
    'day_of_week',
    'month',
    'day_of_month',
    'week_of_year',
    'is_weekend',
    'is_any_holiday',
    'all_holiday_name',
]

In [20]:
train_full = df[['unique_id', 'ds', 'y'] + exog_cols].copy()

In [21]:
frct = MLForecast(
    models=[lgbm],
    lags=lags,
    lag_transforms=lag_transforms,
    freq='D'
)
frct.fit(
    train_full,
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    static_features=[]
)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006252 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3284
[LightGBM] [Info] Number of data points in the train set: 328050, number of used features: 22
[LightGBM] [Info] Start training from score 1.797288


MLForecast(models=[LGBMRegressor], freq=D, lag_features=['lag1', 'lag2', 'lag3', 'lag7', 'lag14', 'lag21', 'lag28', 'lag35', 'lag56', 'rolling_mean_lag1_window_size3', 'rolling_mean_lag1_window_size7', 'rolling_mean_lag1_window_size28', 'rolling_mean_lag1_window_size56', 'rolling_std_lag1_window_size7', 'rolling_std_lag1_window_size28'], date_features=[], num_threads=1)

In [22]:
future = frct.make_future_dataframe(h=7)

# календарні фічі
future['day_of_week']  = future['ds'].dt.dayofweek
future['month']        = future['ds'].dt.month
future['day_of_month'] = future['ds'].dt.day
future['year']         = future['ds'].dt.year
future['week_of_year'] = future['ds'].dt.isocalendar().week.astype('int16')
future['is_weekend']   = future['day_of_week'].isin([5, 6]).astype('int8')

# future['is_observed'] = 0

us_holidays = holidays.US(years=[2016])
future_dates = future['ds'].dt.date
future['us_holiday_name'] = future_dates.map(us_holidays.get)
future['is_any_holiday'] = future['us_holiday_name'].notna().astype('int8')
future['all_holiday_name'] = future['us_holiday_name'].fillna('None').astype('category')

X_future = future[['unique_id', 'ds'] + exog_cols]

pred_7d = frct.predict(h=7, X_df=X_future)


pred_col = [c for c in pred_7d.columns if c not in ['unique_id', 'ds']][0]

submission = (
    pred_7d
    .groupby('unique_id', as_index=False)[pred_col]
    .sum()
    .rename(columns={pred_col: 'y'})
)

submission['y'] = submission['y'].clip(lower=0).round().astype('int64')


In [23]:
submission

,unique_id,y
0,0_FOODS_1_0,23
1,0_FOODS_1_1,16
2,0_FOODS_1_10,29
3,0_FOODS_1_11,21
4,0_FOODS_1_13,63
...,...,...
400,3_HOUSEHOLD_2_169,4
401,3_HOUSEHOLD_2_171,4
402,3_HOUSEHOLD_2_177,2
403,3_HOUSEHOLD_2_179,6


In [24]:
sub = submission.rename(columns={'unique_id': 'index'})
sub.to_csv(data_dir / "submission.csv", index=False)
print("Saved:", sub.shape, sub.columns.tolist())

Saved: (405, 2) ['index', 'y']
